# Dataset To SQL

In [82]:
import pandas as pd

In [83]:
%pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [84]:
import mysql.connector
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='markcelemen1',
    database='customer_shopping_data'
)
cursor = connection.cursor() 
cursor

In [85]:
create_retail_table = """
CREATE TABLE IF NOT EXISTS retail (
    invoice_no VARCHAR(10) PRIMARY KEY,
    customer_id VARCHAR(10),
    gender VARCHAR(10),
    age INT,
    category VARCHAR(20),
    quantity INT,
    price DECIMAL(8, 2),
    payment_method VARCHAR(20),
    invoice_date DATE,
    shopping_mall VARCHAR(20),
    age_group VARCHAR(20),
    invoice_year INT,
    invoice_month INT,
    invoice_day INT,
    price_usd DECIMAL(8,2),
    revenue DECIMAL(8,2)
);
"""
cursor.execute(create_retail_table)
connection.commit()

In [86]:
retail_df = pd.read_csv('preprocessed_customer_shopping_data.csv')
retail_df = retail_df.where(pd.notnull(retail_df), None)

for index, row in retail_df.iterrows():
    insert_retail = """
    INSERT IGNORE INTO retail (invoice_no, customer_id, gender, age, category, quantity, price, payment_method, invoice_date, shopping_mall, age_group, invoice_year, invoice_month, invoice_day, price_usd, revenue)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_retail, (
        row['invoice_no'], row['customer_id'], row['gender'], row['age'],
        row['category'], row['quantity'], row['price'], row['payment_method'],
        row['invoice_date'], row['shopping_mall'], row['age_group'], row['invoice_year'],
        row['invoice_month'], row['invoice_day'], row['price_usd'], row['revenue']
    ))

# Print results to verify insertion
connection.commit()
test_query = "SELECT * FROM retail LIMIT 5"
cursor.execute(test_query)

result = cursor.fetchall()
for row in result:
    print(row)

# Close cursor and connection
cursor.close()
connection.close()

('I100008', 'C199951', 'Male', 65, 'Clothing', 5, Decimal('99.99'), 'Cash', datetime.date(2022, 7, 10), 'Emaar Square Mall', 'Baby Boomer', 2022, 7, 10, Decimal('36.20'), Decimal('99.99'))
('I100014', 'C138893', 'Male', 55, 'Cosmetics', 5, Decimal('99.99'), 'Cash', datetime.date(2021, 6, 18), 'Viaport Outlet', 'Gen X', 2021, 6, 18, Decimal('4.90'), Decimal('24.50'))
('I100015', 'C132779', 'Female', 35, 'Clothing', 2, Decimal('99.99'), 'Debit Card', datetime.date(2021, 3, 4), 'Mall of Istanbul', 'Millennial', 2021, 3, 4, Decimal('14.48'), Decimal('28.96'))
('I100024', 'C244411', 'Female', 67, 'Books', 3, Decimal('45.45'), 'Credit Card', datetime.date(2023, 1, 5), 'Emaar Square Mall', 'Baby Boomer', 2023, 1, 5, Decimal('1.10'), Decimal('3.30'))
('I100027', 'C150002', 'Female', 19, 'Technology', 4, Decimal('99.99'), 'Cash', datetime.date(2022, 5, 18), 'Mall of Istanbul', 'Gen Z', 2022, 5, 18, Decimal('99.99'), Decimal('99.99'))


In [81]:
retail_df[retail_df['invoice_no'] == 'I100008']

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall,age_group,invoice_year,invoice_month,invoice_day,price_usd,revenue
98685,I100008,C199951,Male,65,Clothing,5,1500.4,Cash,2022-07-10,Emaar Square Mall,Baby Boomer,2022,7,10,36.2,181.0
